# 5.3.3 Creating Your First Serverless Distributed SQL Database - Amazon DSQL Cluster

<div style="background-color: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; padding: 10px; margin: 10px;">
<strong>📋 Workshop Contents</strong>
<ul style="line-height: 1.2;">
<li><a href="#What-Well-Build">What We'll Build</a></li>
<li><a href="#Cost-Overview">Cost Overview</a></li>
<li><a href="#Best-Practices">Best Practices</a></li>
<li><a href="#Prerequisites">Prerequisites</a></li>
<li><a href="#Advanced-Application-Multi-Region-Rewards-System">Advanced Application: Multi-Region Rewards System</a></li>
<li><a href="#Visual-Guide-to-Create-Multi-Region-Amazon-DSQL-Cluster">Visual Guide to Create Multi-Region Amazon DSQL Cluster</a></li>
<li><a href="#Step-1:-Check-IAM-Permissions">Step 1: Check IAM Permissions</a></li>
<li><a href="#Step-2:-Create-Multi-Region-Amazon-DSQL-Cluster">Step 2: Create Multi-Region Amazon DSQL Cluster</a></li>
<li><a href="#Step-3:-Check-Pre-installed-Dependencies">Step 3: Check Pre-installed Dependencies</a></li>
<li><a href="#Step-4:-Connect-to-Amazon-Aurora-DSQL">Step 4: Connect to Amazon Aurora DSQL</a></li>
<li><a href="#Step-5:-Create-Schema-and-Load-Sample-Data">Step 5: Create Schema and Load Sample Data</a></li>
<li><a href="#Step-6:-Test-Queries-with-Sample-Data">Step 6: Test Queries with Sample Data</a></li>
<li><a href="#Summary-and-Next-Steps">Summary and Next Steps</a></li>
<li><a href="#Additional-Resources-📚">Additional Resources</a></li>
</ul>
</div>

Let's create a multi-region Amazon DSQL cluster step by step! 🚀

> **💡 What is Amazon Aurora DSQL?**: [Amazon Aurora DSQL](https://aws.amazon.com/rds/aurora/dsql/) is a serverless, distributed SQL database that provides strong consistency across multiple regions with PostgreSQL compatibility. It automatically scales to handle your workload and provides built-in high availability.

## What We'll Build

- **Multi-Region Setup**: Amazon DSQL clusters in us-east-1 and us-east-2
- **Active-Active Configuration**: Strong consistency across all regions
- **Sample Application**: Retail rewards points system with PostgreSQL compatibility
- **IAM Authentication**: Secure connections without managing passwords

## Cost Overview 💰

| Component | Cost | Notes |
|-----------|------|-------|
| Amazon DSQL | Pay-per-use | No minimum charges, scales to zero |
| Data Transfer | Standard rates | Cross-region replication included |
| Storage | Pay for what you use | Automatic scaling and optimization |

💡 **Cost Optimization Tips:**
- Amazon DSQL automatically scales to zero when not in use
- No capacity planning required - pay only for actual usage
- Built-in multi-region replication without additional charges
- Serverless architecture eliminates idle resource costs

## Best Practices ✅

1. **Multi-Region Design**
   - Deploy in multiple regions for high availability
   - Use local endpoints for optimal performance
   - Leverage strong consistency guarantees

2. **Security**
   - Use IAM authentication for secure access
   - Enable encryption in transit and at rest
   - Follow principle of least privilege

Let's get started! 🚀

## Prerequisites

Before starting this tutorial, ensure you have:

- **AWS Account**: Active AWS account with appropriate permissions
- **AWS CLI**: Configured with credentials and default region
- **IAM Permissions**: Required Amazon DSQL permissions (we'll verify these in Step 1)
- **Jupyter Notebook**: You can launch a [free tier Amazon SageMaker Jupyter Notebook](../../1_Getting_Started_with_AWS/1.4_Setting_up_Your_Cookbook_Environment/README.MD)
- **Python Environment**: Python 3.6+ with pip for installing packages
- **Basic Knowledge**: Familiarity with SQL and AWS services

### Required IAM Permissions

Your IAM user or role needs the following permissions:
- `dsql:*` - Full Amazon DSQL permissions
- `[RDS:GenerateDbAuthToken` - For IAM database authentication
- `sts:GetCallerIdentity` - To verify your AWS identity

> **💡 Tip**: If you completed earlier modules in this cookbook, you likely already have the necessary permissions configured.

## Advanced Application: Multi-Region Rewards System

<div style="padding: 15px; background-color: #e6f7e6; border-left: 5px solid #28a745; margin-bottom: 10px;">
<strong>🚀 Advanced Use Case:</strong> You can enhance the serverless web application from <a href="../../3_Building_Your_First_Serverless_Web_App_with_Aurora/README.MD">Module 3: Building Your First Serverless Web App with Aurora</a> by replacing the Aurora database with Amazon DSQL. This upgrade transforms your application into a multi-region resilient system that can withstand regional failures while maintaining strong data consistency across regions.
</div>

### Application Architecture

The retail rewards points system demonstrates Amazon DSQL's capabilities:

- **Frontend**: AWS Amplify hosting the web interface
- **API Layer**: API Gateway with AWS Lambda functions
- **Authentication**: Amazon Cognito user pools
- **Database**: Amazon DSQL with multi-region active-active setup
- **Authorization**: IAM for fine-grained access control

### Key Benefits

- **Regional Resilience**: Application continues operating even if one region fails
- **Strong Consistency**: Data remains consistent across all regions
- **Transparent Scaling**: Amazon DSQL handles scaling automatically
- **Cost Optimization**: Pay only for actual usage with serverless architecture

![Retail rewards points system powered by Aurora DSQL](../images/5.3-retail-awards-architecture-aurora-dsql.png)

Now, let's create an Amazon DSQL cluster to power this architecture!

## Visual Guide to Amazon Aurora DSQL Console

![Creating Amazon Aurora DSQL Cluster](../images/5.3-create-aurora-dsql.gif)
*Step-by-step console walkthrough of creating an Amazon DSQL cluster*

> **💡 Note**: You can also follow the step-by-step guidance as follows to create Amazon Aurora DSQL through CLIs. Otherwise, jump to <a href="#Step-4:-Connect-to-Amazon-Aurora-DSQL">Step 4: Connect to Amazon Aurora DSQL</a>.

## Step 1: Check IAM Permissions

First, we'll upgrade the AWS CLI to support DSQL commands, then verify your IAM permissions for Amazon DSQL operations. The SageMaker environment has an older AWS CLI version that doesn't include DSQL support.

In [ ]:
%%bash
echo "🔍 Checking IAM permissions for Aurora DSQL..."

# Check if we can list DSQL clusters (this will test basic permissions)
if aws dsql list-clusters --region us-east-1 > /dev/null 2>&1; then
    echo "✅ Aurora DSQL permissions verified"
else
    echo "❌ Missing Aurora DSQL permissions"
    echo "Required IAM policy:"
    cat << 'EOF'
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "dsql:CreateCluster",
        "dsql:GetCluster",
        "dsql:UpdateCluster",
        "dsql:DeleteCluster",
        "dsql:ListClusters",
        "dsql:CreateMultiRegionClusters",
        "dsql:DeleteMultiRegionClusters",
        "dsql:DbConnect",
        "dsql:DbConnectAdmin"
      ],
      "Resource": "*"
    }
  ]
}
EOF
fi

# Get current identity
echo "Current AWS identity:"
aws sts get-caller-identity --output table

## Step 2: Create Multi-Region Amazon DSQL Cluster

This step creates a true multi-region DSQL cluster with active-active replication between us-east-1 and us-east-2, using us-west-2 as the witness region. The process involves creating two clusters and configuring them as peers for strong consistency across regions.

In [ ]:
%%bash
echo "🚀 Creating multi-region DSQL clusters with proper peering..."

# Create first cluster in us-east-1 with witness region
CLUSTER_1_ARN=$(aws dsql create-cluster \
    --region us-east-1 \
    --tags Name="Rewards Multi-Region" CreationSource="aws-database-cookbook-v2025.8" \
    --multi-region-properties witnessRegion=us-west-2 \
    --query 'arn' --output text)

echo "✅ Created cluster 1: $CLUSTER_1_ARN"
CLUSTER_1_ID=$(echo $CLUSTER_1_ARN | cut -d'/' -f2)

# Create second cluster in us-east-2 with witness region and peer
CLUSTER_2_ARN=$(aws dsql create-cluster \
    --region us-east-2 \
    --tags Name="Rewards Multi-Region" CreationSource="aws-database-cookbook-v2025.8" \
    --multi-region-properties witnessRegion=us-west-2,clusters=[$CLUSTER_1_ARN] \
    --query 'arn' --output text)

echo "✅ Created cluster 2: $CLUSTER_2_ARN"
CLUSTER_2_ID=$(echo $CLUSTER_2_ARN | cut -d'/' -f2)

# Update first cluster to peer with second cluster
aws dsql update-cluster \
    --identifier $CLUSTER_1_ID \
    --region us-east-1 \
    --multi-region-properties witnessRegion=us-west-2,clusters=[$CLUSTER_2_ARN]

echo "🔗 Peered clusters for multi-region setup"

# Wait for clusters to become active
echo "⏳ Waiting for clusters to become ACTIVE..."

while true; do
    STATUS_1=$(aws dsql get-cluster --identifier $CLUSTER_1_ID --region us-east-1 --query 'status' --output text)
    STATUS_2=$(aws dsql get-cluster --identifier $CLUSTER_2_ID --region us-east-2 --query 'status' --output text)
    echo "Cluster 1 status: $STATUS_1, Cluster 2 status: $STATUS_2"
    if [ "$STATUS_1" = "ACTIVE" ] && [ "$STATUS_2" = "ACTIVE" ]; then
        break
    fi
    sleep 60
done

# Save cluster information
echo "export PRIMARY_CLUSTER_ARN=$CLUSTER_1_ARN" > .dsql_cluster_info
echo "export PRIMARY_CLUSTER_ID=$CLUSTER_1_ID" >> .dsql_cluster_info
echo "export SECONDARY_CLUSTER_ARN=$CLUSTER_2_ARN" >> .dsql_cluster_info
echo "export SECONDARY_CLUSTER_ID=$CLUSTER_2_ID" >> .dsql_cluster_info
echo "export PRIMARY_ENDPOINT=$CLUSTER_1_ID.dsql.us-east-1.on.aws" >> .dsql_cluster_info
echo "export SECONDARY_ENDPOINT=$CLUSTER_2_ID.dsql.us-east-2.on.aws" >> .dsql_cluster_info

echo "✅ Multi-region DSQL clusters are now ACTIVE!"
echo "Primary: $CLUSTER_1_ID.dsql.us-east-1.on.aws"
echo "Secondary: $CLUSTER_2_ID.dsql.us-east-2.on.aws"

### Multi-Region Configuration Summary

The multi-region DSQL cluster provides:

In [ ]:
%%bash
# Display actual cluster configuration
source .dsql_cluster_info

echo "📋 Multi-Region DSQL Configuration:"
echo "- Primary Region: us-east-1 (N.Virginia)"
echo "- Secondary Region: us-east-2 (Ohio)"
echo "- Witness Region: us-west-2 (Oregon)"
echo "- Architecture: Active-Active Multi-Region"
echo "- Consistency: Strong consistency across all regions"

echo "🔍 Cluster Details:"
aws dsql get-cluster --identifier $PRIMARY_CLUSTER_ID --region us-east-1 --query '{Status:status,Arn:arn}' --output table
aws dsql get-cluster --identifier $SECONDARY_CLUSTER_ID --region us-east-2 --query '{Status:status,Arn:arn}' --output table

## Step 3: Check Pre-installed Dependencies

The SageMaker notebook environment deployed by the CloudFormation template includes pre-installed packages. Let's verify the availability of required dependencies instead of installing them directly. The SageMaker lifecycle configuration automatically installs psycopg[binary], boto3, and pandas during notebook startup.

In [ ]:
import sys
import subprocess

def check_package(package_name):
    """Check if a package is installed"""
    try:
        __import__(package_name)
        return True
    except ImportError:
        return False

print("📦 Checking pre-installed dependencies from SageMaker environment...")

# Check required packages
packages = {
    'boto3': 'AWS SDK for Python',
    'psycopg': 'PostgreSQL adapter for Python',
    'pandas': 'Data manipulation and analysis library'
}

all_available = True
for package, description in packages.items():
    if check_package(package):
        print(f"✅ {package}: {description} - Available")
    else:
        print(f"❌ {package}: {description} - Not available")
        all_available = False

if all_available:
    print("✅ All required dependencies are pre-installed in the SageMaker environment!")
else:
    print("⚠️ Some packages are missing. Installing now...")
    subprocess.run([sys.executable, '-m', 'pip', 'install', '--quiet', 'psycopg[binary]', 'boto3', 'pandas'])
    print("✅ Missing dependencies installed successfully")

## Step 4: Connect to Amazon Aurora DSQL

The following script defines essential Python functions for Amazon DSQL connectivity, including loading cluster information from the saved file, generating IAM authentication tokens, and establishing secure database connections. The functions handle error cases gracefully and provide clear status messages during connection attempts. These utilities will be used throughout the remaining steps for all database operations.

In [ ]:
import boto3
import psycopg
import json
import os
import pandas as pd
from datetime import datetime

# Load cluster information
def load_cluster_info():
    """Load cluster information from environment file"""
    cluster_info = {}
    try:
        with open('.dsql_cluster_info', 'r') as f:
            for line in f:
                if line.startswith('export '):
                    key, value = line.replace('export ', '').strip().split('=', 1)
                    cluster_info[key] = value
    except FileNotFoundError:
        print("❌ Cluster info file not found. Please run the cluster creation steps first.")
        return None
    return cluster_info

# Get IAM authentication token
def get_iam_token(endpoint, region):
    """Generate IAM authentication token for Aurora DSQL"""
    try:
        # Use DSQL client for admin token generation
        dsql_client = boto3.client('dsql', region_name=region)
        token = dsql_client.generate_db_connect_admin_auth_token(endpoint, region)
        return token
    except Exception as e:
        print(f"❌ Error generating IAM token: {e}")
        return None

# Connect to Aurora DSQL
def connect_to_dsql(endpoint, region, database='postgres'):
    """Connect to Aurora DSQL using IAM authentication"""
    print(f"🔌 Connecting to Aurora DSQL at {endpoint}...")
    
    # Use admin user for DSQL connection
    try:
        username = 'admin'
        print(f"Using DSQL admin user: {username}")
        
        # Get IAM token
        token = get_iam_token(endpoint, region)
        if not token:
            return None
        
        # Try connection with IAM authentication
        conn = psycopg.connect(
            host=endpoint,
            port=5432,
            dbname=database,
            user=username,
            password=token,
            sslmode='require',
            connect_timeout=30
        )
        print(f"✅ Successfully connected to {region}")
        return conn
        
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return None

print("🔧 Connection functions defined successfully")

## Step 5: Create Schema and Load Sample Data

This step creates a retail rewards database schema and loads sample data into your Amazon DSQL cluster. We'll create tables for customers, products, orders, and rewards to demonstrate cross-region replication capabilities.

In [ ]:
# Create directory for SQL files if it doesn't exist
import os

# Create sql directory if it doesn't exist
os.makedirs('./sql', exist_ok=True)

# Create schema file if it doesn't exist
schema_file = './sql/aurora_dsql_schema.sql'
if not os.path.exists(schema_file):
    print(f"Creating schema file: {schema_file}")
    with open(schema_file, 'w') as f:
        f.write('''
-- Create schema for retail rewards application
CREATE SCHEMA IF NOT EXISTS xpoints;

-- Create customers table
CREATE TABLE IF NOT EXISTS xpoints.customers (
    customer_id VARCHAR(36) PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL,
    phone VARCHAR(20),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    region VARCHAR(20) NOT NULL
);

-- Create products table
CREATE TABLE IF NOT EXISTS xpoints.products (
    product_id VARCHAR(36) PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    description TEXT,
    price DECIMAL(10, 2) NOT NULL,
    category VARCHAR(50) NOT NULL,
    inventory_count INT NOT NULL DEFAULT 0,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    region VARCHAR(20) NOT NULL
);

-- Create orders table
CREATE TABLE IF NOT EXISTS xpoints.orders (
    order_id VARCHAR(36) PRIMARY KEY,
    customer_id VARCHAR(36) NOT NULL REFERENCES xpoints.customers(customer_id),
    order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    status VARCHAR(20) NOT NULL,
    total_amount DECIMAL(10, 2) NOT NULL,
    shipping_address TEXT,
    region VARCHAR(20) NOT NULL,
    CONSTRAINT valid_status CHECK (status IN (''pending'', ''processing'', ''shipped'', ''delivered'', ''cancelled''))
);

-- Create order_items table
CREATE TABLE IF NOT EXISTS xpoints.order_items (
    order_item_id VARCHAR(36) PRIMARY KEY,
    order_id VARCHAR(36) NOT NULL REFERENCES xpoints.orders(order_id),
    product_id VARCHAR(36) NOT NULL REFERENCES xpoints.products(product_id),
    quantity INT NOT NULL,
    price_per_unit DECIMAL(10, 2) NOT NULL
);

-- Create rewards table
CREATE TABLE IF NOT EXISTS xpoints.rewards (
    reward_id VARCHAR(36) PRIMARY KEY,
    customer_id VARCHAR(36) NOT NULL REFERENCES xpoints.customers(customer_id),
    points_balance INT NOT NULL DEFAULT 0,
    last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    region VARCHAR(20) NOT NULL
);

-- Create reward_transactions table
CREATE TABLE IF NOT EXISTS xpoints.reward_transactions (
    transaction_id VARCHAR(36) PRIMARY KEY,
    reward_id VARCHAR(36) NOT NULL REFERENCES xpoints.rewards(reward_id),
    points_change INT NOT NULL,
    transaction_type VARCHAR(20) NOT NULL,
    transaction_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    order_id VARCHAR(36) REFERENCES xpoints.orders(order_id),
    notes TEXT,
    region VARCHAR(20) NOT NULL,
    CONSTRAINT valid_transaction_type CHECK (transaction_type IN (''earn'', ''redeem'', ''expire'', ''adjust''))
);
''')
    print(f"✅ Schema file created: {schema_file}")
else:
    print(f"✅ Schema file already exists: {schema_file}")

print("✅ SQL files prepared!")

Now let's create the schema and load initial data into the primary region (us-east-1):

In [ ]:
%%bash
echo "🏗️ Creating schema and loading data..."

# Source cluster info
source .dsql_cluster_info

# Get DSQL admin token using AWS CLI
TOKEN=$(aws dsql generate-db-connect-admin-auth-token --hostname $PRIMARY_ENDPOINT --region us-east-1 --output text)
echo "✅ Successfully generated authentication token"

# Set connection variables
export PGHOST=$PRIMARY_ENDPOINT
export PGPORT=5432
export PGDATABASE=postgres
export PGUSER=admin
export PGPASSWORD=$TOKEN
export PGSSLMODE=require

# Create schema
echo "Creating schema..."
psql -f ./sql/aurora_dsql_schema.sql

echo "✅ Schema creation complete!"

# Insert sample data into us-east-1 (primary region)
echo "🌟 Inserting sample data in us-east-1 region..."
psql << EOF
-- Insert customers
INSERT INTO xpoints.customers (customer_id, first_name, last_name, email, phone, region)
VALUES 
('c1', 'John', 'Smith', 'john.smith@example.com', '555-123-4567', 'us-east-1'),
('c2', 'Jane', 'Doe', 'jane.doe@example.com', '555-234-5678', 'us-east-1'),
('c3', 'Robert', 'Johnson', 'robert.j@example.com', '555-345-6789', 'us-east-1');

-- Insert products
INSERT INTO xpoints.products (product_id, name, description, price, category, inventory_count, region)
VALUES 
('p1', 'Wireless Earbuds', 'High-quality wireless earbuds with noise cancellation', 129.99, 'Electronics', 50, 'us-east-1'),
('p2', 'Smart Watch', 'Fitness tracking smartwatch with heart rate monitor', 199.99, 'Electronics', 30, 'us-east-1'),
('p3', 'Portable Charger', '10000mAh portable power bank', 49.99, 'Accessories', 100, 'us-east-1');

-- Insert orders
INSERT INTO xpoints.orders (order_id, customer_id, status, total_amount, shipping_address, region)
VALUES 
('o1', 'c1', 'delivered', 129.99, '123 Main St, Anytown, USA', 'us-east-1'),
('o2', 'c2', 'shipped', 249.98, '456 Oak Ave, Somewhere, USA', 'us-east-1');

-- Insert order items
INSERT INTO xpoints.order_items (order_item_id, order_id, product_id, quantity, price_per_unit)
VALUES 
('oi1', 'o1', 'p1', 1, 129.99),
('oi2', 'o2', 'p1', 1, 129.99),
('oi3', 'o2', 'p3', 1, 49.99);

-- Insert rewards
INSERT INTO xpoints.rewards (reward_id, customer_id, points_balance, region)
VALUES 
('r1', 'c1', 130, 'us-east-1'),
('r2', 'c2', 250, 'us-east-1'),
('r3', 'c3', 0, 'us-east-1');

-- Insert reward transactions
INSERT INTO xpoints.reward_transactions (transaction_id, reward_id, points_change, transaction_type, order_id, notes, region)
VALUES 
('t1', 'r1', 130, 'earn', 'o1', 'Points for order #o1', 'us-east-1'),
('t2', 'r2', 250, 'earn', 'o2', 'Points for order #o2', 'us-east-1');

EOF

echo "✅ Sample data inserted in us-east-1 region!"

## Step 6: Test Queries with Sample Data

This step demonstrates the cross-region replication capabilities of Amazon DSQL. We'll insert data in one region, query it from another region, update data in the second region, and verify that changes are immediately visible in the first region - all with strong consistency.

In [ ]:
def test_cross_region_replication():
    """Test cross-region replication by inserting data in one region and querying from another"""
    # Load cluster information
    cluster_info = load_cluster_info()
    if not cluster_info:
        return
    
    # Get endpoints for both regions
    primary_endpoint = cluster_info.get('PRIMARY_ENDPOINT')
    secondary_endpoint = cluster_info.get('SECONDARY_ENDPOINT')
    
    print("🔄 Testing cross-region replication...")
    print("\n1️⃣ STEP 1: Query data from secondary region (us-east-2) to verify initial replication")
    
    # Connect to secondary region
    conn_secondary = connect_to_dsql(secondary_endpoint, 'us-east-2')
    if not conn_secondary:
        return
    
    try:
        # Query customers from secondary region
        cursor = conn_secondary.cursor()
        cursor.execute("SELECT customer_id, first_name, last_name, email, region FROM xpoints.customers ORDER BY customer_id")
        print("\n📋 Customers in us-east-2 (should show data inserted in us-east-1):")
        results = cursor.fetchall()
        for row in results:
            print(f"   {row}")
        
        # Query rewards from secondary region
        cursor.execute("SELECT reward_id, customer_id, points_balance, region FROM xpoints.rewards ORDER BY reward_id")
        print("\n📋 Rewards in us-east-2 (should show data inserted in us-east-1):")
        results = cursor.fetchall()
        for row in results:
            print(f"   {row}")
        
        print("\n2️⃣ STEP 2: Insert new data in secondary region (us-east-2)")
        
        # Insert new customer in secondary region
        cursor.execute("""
        INSERT INTO xpoints.customers (customer_id, first_name, last_name, email, phone, region)
        VALUES ('c4', 'Emily', 'Wilson', 'emily.w@example.com', '555-456-7890', 'us-east-2')
        RETURNING customer_id, first_name, last_name, email, region
        """)
        new_customer = cursor.fetchone()
        print(f"\n✅ New customer inserted in us-east-2: {new_customer}")
        
        # Insert new product in secondary region
        cursor.execute("""
        INSERT INTO xpoints.products (product_id, name, description, price, category, inventory_count, region)
        VALUES ('p4', 'Bluetooth Speaker', 'Waterproof portable speaker', 79.99, 'Electronics', 45, 'us-east-2')
        RETURNING product_id, name, price, region
        """)
        new_product = cursor.fetchone()
        print(f"✅ New product inserted in us-east-2: {new_product}")
        
        # Insert new reward in secondary region
        cursor.execute("""
        INSERT INTO xpoints.rewards (reward_id, customer_id, points_balance, region)
        VALUES ('r4', 'c4', 100, 'us-east-2')
        RETURNING reward_id, customer_id, points_balance, region
        """)
        new_reward = cursor.fetchone()
        print(f"✅ New reward inserted in us-east-2: {new_reward}")
        
        # Commit changes
        conn_secondary.commit()
        cursor.close()
        conn_secondary.close()
        
        print("\n3️⃣ STEP 3: Query data from primary region (us-east-1) to verify replication from us-east-2")
        
        # Connect to primary region
        conn_primary = connect_to_dsql(primary_endpoint, 'us-east-1')
        if not conn_primary:
            return
        
        cursor = conn_primary.cursor()
        
        # Query customers from primary region
        cursor.execute("SELECT customer_id, first_name, last_name, email, region FROM xpoints.customers WHERE customer_id = 'c4'")
        result = cursor.fetchone()
        print(f"\n📋 Customer c4 in us-east-1 (should show data inserted in us-east-2): {result}")
        
        # Query products from primary region
        cursor.execute("SELECT product_id, name, price, region FROM xpoints.products WHERE product_id = 'p4'")
        result = cursor.fetchone()
        print(f"📋 Product p4 in us-east-1 (should show data inserted in us-east-2): {result}")
        
        # Query rewards from primary region
        cursor.execute("SELECT reward_id, customer_id, points_balance, region FROM xpoints.rewards WHERE reward_id = 'r4'")
        result = cursor.fetchone()
        print(f"📋 Reward r4 in us-east-1 (should show data inserted in us-east-2): {result}")
        
        print("\n4️⃣ STEP 4: Update data in primary region (us-east-1) and verify in secondary region (us-east-2)")
        
        # Update reward points in primary region
        cursor.execute("""
        UPDATE xpoints.rewards 
        SET points_balance = points_balance + 50 
        WHERE reward_id = 'r4'
        RETURNING reward_id, customer_id, points_balance, region
        """)
        updated_reward = cursor.fetchone()
        print(f"\n✅ Updated reward in us-east-1: {updated_reward}")
        
        # Commit changes
        conn_primary.commit()
        cursor.close()
        conn_primary.close()
        
        # Connect back to secondary region to verify update
        conn_secondary = connect_to_dsql(secondary_endpoint, 'us-east-2')
        if not conn_secondary:
            return
        
        cursor = conn_secondary.cursor()
        cursor.execute("SELECT reward_id, customer_id, points_balance, region FROM xpoints.rewards WHERE reward_id = 'r4'")
        result = cursor.fetchone()
        print(f"📋 Reward r4 in us-east-2 (should show updated points from us-east-1): {result}")
        
        cursor.close()
        conn_secondary.close()
        
        print("\n✅ Cross-region replication test completed successfully!")
        print("✅ Changes made in one region are immediately visible in the other region")
        print("✅ This demonstrates Aurora DSQL's strong consistency across regions")
        
    except Exception as e:
        print(f"❌ Error during cross-region replication test: {e}")
        if conn_secondary and not conn_secondary.closed:
            conn_secondary.close()

# Run the cross-region replication test
test_cross_region_replication()

## Summary and Next Steps

🎉 **Congratulations!** You have successfully:

✅ Created multi-region Amazon DSQL clusters in us-east-1 and us-east-2  
✅ Established active-active replication between regions  
✅ Connected to both clusters using IAM authentication  
✅ Created the rewards application schema with sample data  
✅ Tested queries across both regions  
✅ Demonstrated real-time cross-region replication with strong consistency

### Key Benefits Demonstrated

- **Serverless**: No capacity planning required - scales automatically
- **PostgreSQL Compatible**: Use familiar SQL syntax and tools
- **IAM Authentication**: Secure connections without managing passwords
- **Multi-region active-active replication**: Changes from 1 region are seamlessly synchronized to another region

### Connection Information Summary

Your cluster connection details have been saved to `.dsql_cluster_info`. Use these endpoints in your applications for PostgreSQL-compatible access to your distributed SQL database.

### Next Steps 🚀

1. **Explore Amazon Aurora DSQL Workshop**: You will interact with the Aurora DSQL database and run some application code from [AWS CloudShell](https://docs.aws.amazon.com/cloudshell/latest/userguide/welcome.html). CloudShell is a browser-based Linux shell that you can launch directly from the AWS Management Console. You won't have to install any software on your workstation to complete this workshop.
2. **Create schema and load data**: After [preparing your Amazon Aurora DSQL workshop environment](https://catalog.workshops.aws/aurora-dsql/en-US/01-getting-started/01-set-up-first-region), you can create schemas and load some data that later on for you to enhance your Xanadu Rewards Application built in [Section 3. Building Your First Serverless Web App with Amazon Aurora](../../3_Building_Your_First_Serverless_Web_App_with_Aurora/README.MD) with a multi-region active-active synchronization capability. With Aurora DSQL's bi-directional synchronous replication, your application can now accept reads and writes in two AWS regions.
3. **Explore the Data Model**: Continue with the [Amazon Amazon DSQL Workshop - Explore the Data Model](https://catalog.workshops.aws/aurora-dsql/en-US/02-working-with-aurora-dsql/04-explore-the-data-model) to learn advanced querying techniques and data modeling best practices.
4. **Build Applications**: [Integrate Amazon DSQL into your Reward System application](https://catalog.workshops.aws/aurora-dsql/en-US/03-building-the-rewards-app) using the connection patterns demonstrated above.
5. **Explore Migration Options**: Learn how to migrate existing PostgreSQL databases to Amazon DSQL in [5.3.3 Migrating to Amazon Aurora DSQL](./5.3.3_Migrating-to-Aurora-DSQL.ipynb)

🎉 **Excellent!** You've successfully set up multi-region Amazon DSQL clusters and learned the fundamentals of distributed SQL databases. Your foundation for building globally distributed applications is now in place!

**Ready to continue?** Let's advance to [6. Optimizing Performance and Cost: Mastering Aurora Efficiency](../../6_Optimizing_Performance_and_Cost/README.md) and how to fine-tune your Aurora database for maximum performance and cost-effectiveness!

### Cleanup Resources 🧹

When you're done experimenting, remember to clean up your resources to avoid ongoing charges:

In [ ]:
# This cleanup script deletes both Aurora DSQL clusters to avoid ongoing charges once you're finished with the tutorial.
# The commands are commented out for safety - uncomment them when you're ready to permanently delete the clusters.
# You'll see deletion confirmation messages for both the secondary and primary clusters.

%%bash
# Uncomment and run this cell to clean up resources
# echo "🧹 Cleaning up Aurora DSQL resources..."
# 
# # Source cluster info
# source .dsql_cluster_info
# 
# # Delete both clusters
# echo "Deleting secondary cluster..."
# aws dsql delete-cluster --identifier $SECONDARY_CLUSTER_ID --region us-east-2
# 
# echo "Deleting primary cluster..."
# aws dsql delete-cluster --identifier $PRIMARY_CLUSTER_ID --region us-east-1
# 
# echo "✅ Cleanup initiated. Both clusters will be deleted shortly."

echo "💡 Uncomment the lines above to clean up your Aurora DSQL clusters"

## Additional Resources 📚

### Aurora Features
- [Amazon Aurora PostgreSQL Limitless Database supports CloudWatch Database Insights](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/limitless-monitoring.cwdbi.html)
- [Amazon Aurora PostgreSQL zero-ETL integrations support additional Regions](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Concepts.Aurora_Fea_Regions_DB-eng.Feature.Zero-ETL.html)
- [Amazon Aurora PostgreSQL Limitless Database](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/limitless.html)

### Serverless & Scaling
- [End-of-life information for Amazon Aurora Serverless v1](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/[Aurora]-serverless-v2.upgrade.html#[Aurora]-serverless-v2.upgrade-from-serverless-v1-procedure)
- [Amazon Aurora Serverless v2 supports automatic pause and resume](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/[Aurora]-serverless-v2-auto-pause.html)
- [Amazon Aurora Serverless v2 supports 256 ACUs](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/[Aurora]-serverless-v2.how-it-works.html#[Aurora]-serverless-v2.how-it-works.capacity)

### Security & Authentication
- [Amazon RDS support for AWS Secrets Manager Integration]()
